In [1]:
from sqlite3 import SQLITE_CREATE_VIEW
import anndata
import scanpy as sc
import os
from bbknn import bbknn
import matplotlib.pyplot as plt
import omicverse as ov
import scvi
import scib
from scvi.model.utils import mde
from scib_metrics.benchmark import Benchmarker
import scanorama
import numpy as np
import pyliger
from harmony import harmonize
from rich import print
import seaborn as sns
import bbknn
import doubletdetection
%matplotlib inline
plt.rcParams.update({'font.size': 12})


   ____            _     _    __                  
  / __ \____ ___  (_)___| |  / /__  _____________ 
 / / / / __ `__ \/ / ___/ | / / _ \/ ___/ ___/ _ \ 
/ /_/ / / / / / / / /__ | |/ /  __/ /  (__  )  __/ 
\____/_/ /_/ /_/_/\___/ |___/\___/_/  /____/\___/                                              

Version: 1.5.3, Tutorials: https://omicverse.readthedocs.io/


Global seed set to 0
/home/lanxiang/miniconda3/envs/sc/lib/python3.8/site-packages/umap/plot.py:203: NumbaDeprecationWarning: The keyword argument 'nopython=False' was supplied. From Numba 0.59.0 the default is being changed to True and use of 'nopython=False' will raise a warning as the argument will have no effect. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.


In [2]:
# R interface
from rpy2.robjects import pandas2ri
from rpy2.robjects import r
import rpy2.rinterface_lib.callbacks
import anndata2ri

pandas2ri.activate()
anndata2ri.activate()

%load_ext rpy2.ipython

In [3]:
%%R
# R packages
library(Seurat)


    an issue that caused a segfault when used with rpy2:
    https://github.com/rstudio/reticulate/pull/1188
    Make sure that you use a version of that package that includes
    the fix.
    

Loading required package: SeuratObject
Loading required package: sp

Attaching package: ‘SeuratObject’

The following object is masked from ‘package:base’:

    intersect

In addition: Warning message:
In (function (package, help, pos = 2, lib.loc = NULL, character.only = FALSE,  :
  library ‘/usr/lib/R/site-library’ contains no packages


In [4]:
#####Change path and Load data
os.chdir("/mnt/disk5/Lanxiang/Human_integrated/integrated/8_Progress/monkey_ref_all")
adata = anndata.read_h5ad("monkey_remove_Sasaki.h5ad")

/home/lanxiang/miniconda3/envs/sc/lib/python3.8/site-packages/anndata/_io/utils.py:202: OldFormatWarning: Element '/obs/orig.ident' was written without encoding metadata.
/home/lanxiang/miniconda3/envs/sc/lib/python3.8/site-packages/anndata/_io/utils.py:202: OldFormatWarning: Element '/obs/nCount_RNA' was written without encoding metadata.
/home/lanxiang/miniconda3/envs/sc/lib/python3.8/site-packages/anndata/_io/utils.py:202: OldFormatWarning: Element '/obs/nFeature_RNA' was written without encoding metadata.
/home/lanxiang/miniconda3/envs/sc/lib/python3.8/site-packages/anndata/_io/utils.py:202: OldFormatWarning: Element '/obs/sample' was written without encoding metadata.
/home/lanxiang/miniconda3/envs/sc/lib/python3.8/site-packages/anndata/_io/utils.py:202: OldFormatWarning: Element '/obs/stage' was written without encoding metadata.
/home/lanxiang/miniconda3/envs/sc/lib/python3.8/site-packages/anndata/_io/utils.py:202: OldFormatWarning: Element '/obs/percent.mt' was written without 

In [5]:
adata

AnnData object with n_obs × n_vars = 160667 × 51691
    obs: 'orig.ident', 'nCount_RNA', 'nFeature_RNA', 'sample', 'stage', 'percent.mt', 'species', 'embryo', 'platform', 'anno', 'sub_anno', 'harmo.anno'
    var: 'features'

In [6]:
adata.obs

,orig.ident,nCount_RNA,nFeature_RNA,sample,stage,percent.mt,species,embryo,platform,anno,sub_anno,harmo.anno
E13_MS10T45_1_1,Nakamura,999973.322409,10720,2,E13,0.176359,Macaca fascicularis,Real Embryo in vivo,SC3-seq,EXMC,EXMC,ExE.Mesoderm
E13_MS10T46_1_1,Nakamura,999951.611010,10044,2,E13,0.198974,Macaca fascicularis,Real Embryo in vivo,SC3-seq,EXMC,EXMC,ExE.Mesoderm
E13_MS10T48_1_1,Nakamura,999982.757048,10127,2,E13,0.187927,Macaca fascicularis,Real Embryo in vivo,SC3-seq,EXMC,EXMC,ExE.Mesoderm
E13_MS10T49_1_1,Nakamura,999987.594120,9805,2,E13,0.202824,Macaca fascicularis,Real Embryo in vivo,SC3-seq,EXMC,EXMC,ExE.Mesoderm
E13_MS13T66_1_1,Nakamura,999928.974545,11141,2,E13,0.170418,Macaca fascicularis,Real Embryo in vivo,SC3-seq,EXMC,EXMC,ExE.Mesoderm
...,...,...,...,...,...,...,...,...,...,...,...,...
X20210927.SFY.1.5_sc4,Zhai_2023,136376.000000,4440,2,E18_IVC,0.182583,Macaca fascicularis,E18-1,scChaRM-seq,Endoderm,Endoderm,Endoderm
X20210927.SFY.1.5_sc5,Zhai_2023,37566.000000,3267,2,E18_IVC,0.244902,Macaca fascicularis,E18-1,scChaRM-seq,EXMC,EXMC,ExE.Mesoderm
X20210927.SFY.1.5_sc7,Zhai_2023,206313.000000,4915,2,E18_IVC,0.133777,Macaca fascicularis,E18-1,scChaRM-seq,CTB,CTB,TE/TrB
X20210927.SFY.1.5_sc8,Zhai_2023,375795.000000,7880,2,E18_IVC,0.232308,Macaca fascicularis,E18-1,scChaRM-seq,CTB,CTB,TE/TrB


In [7]:
set(adata.obs["harmo.anno"])

{'Endoderm',
 'Epiblast',
 'ExE.Endoderm',
 'ExE.Mesoderm',
 'Hemogenic.Endothelial.Progenitors',
 'Hypoblast',
 'ICM',
 'Mesoderm',
 'Neural.Ectoderm',
 'Node',
 'Non-Neural.Ectoderm',
 'PGC',
 'Primitive.streak',
 'TE/TrB',
 'Unknown'}

In [ ]:
sc.pl.violin(
    adata,
    ["nCount_RNA", "nFeature_RNA", "percent.mt"],
    jitter=0.4,
    multi_panel=True,
    save= "violin_plot.pdf"
)

In [9]:
##filter nFeature_RNA and percent.mt
#adata = adata[adata.obs.nFeature_RNA > 2000, :]
#adata = adata[adata.obs.nCount_RNA > 1000, :]
#adata = adata[adata.obs["percent.mt"] < 15, :]

In [10]:
# remove "empty" genes
sc.pp.filter_genes(adata, min_cells=1)

In [11]:
adata

AnnData object with n_obs × n_vars = 160667 × 44223
    obs: 'orig.ident', 'nCount_RNA', 'nFeature_RNA', 'sample', 'stage', 'percent.mt', 'species', 'embryo', 'platform', 'anno', 'sub_anno', 'harmo.anno'
    var: 'features', 'n_cells'

In [12]:
clf = doubletdetection.BoostClassifier(
    n_iters=40,
    clustering_algorithm="louvain",
    standard_scaling=True,
    pseudocount=0.1,
    n_jobs=-1,
)
doublets = clf.fit(adata.X).predict(p_thresh=1e-16, voter_thresh=0.5)
doublet_score = clf.doublet_score()

  0%|          | 0/40 [00:00<?, ?it/s]

In [13]:
adata.obs["doublet"] = doublets
adata.obs["doublet_score"] = doublet_score

In [14]:
adata

AnnData object with n_obs × n_vars = 160667 × 44223
    obs: 'orig.ident', 'nCount_RNA', 'nFeature_RNA', 'sample', 'stage', 'percent.mt', 'species', 'embryo', 'platform', 'anno', 'sub_anno', 'harmo.anno', 'doublet', 'doublet_score'
    var: 'features', 'n_cells'

In [15]:
print(adata.X)

(0, 0)        10.4088
  (0, 2)        11.8958
  (0, 3)        34.2003
  (0, 26)       7.43484
  (0, 38)       92.1921
  (0, 39)       13.3827
  (0, 41)       5.94788
  (0, 42)       2.97394
  (0, 43)       5.94788
  (0, 44)       2.97394
  (0, 46)       72.8615
  (0, 48)       26.7654
  (0, 49)       5.94788
  (0, 51)       17.8436
  (0, 52)       52.0439
  (0, 54)       20.8176
  (0, 57)       132.34
  (0, 58)       8.92181
  (0, 59)       303.342
  (0, 63)       196.28
  (0, 64)       92.1921
  (0, 67)       89.2181
  (0, 72)       4.46091
  (0, 74)       72.8615
  (0, 76)       59.4788
  :     :
  (160666, 44028)       29.0
  (160666, 44037)       15.0
  (160666, 44038)       696.0
  (160666, 44042)       115.0
  (160666, 44045)       23.0
  (160666, 44073)       44.0
  (160666, 44084)       1.0
  (160666, 44090)       1.0
  (160666, 44094)       45.0
  (160666, 44102)       3.0
  (160666, 44104)       2.0
  (160666, 44111)       35.0
  (160666, 44115)       26.0
  (160666, 44121)       1.0
  (160666, 44139)       1.0
  (160666, 44143)       67.0
  (160666, 44153)       1.0
  (160666, 44171)       5.0
  (160666, 44184)       2.0
  (160666, 44186)       9.0
  (160666, 44191)       1.0
  (160666, 44199)       1.0
  (160666, 44207)       23.0
  (160666, 44211)       85.0
  (160666, 44218)       14.0

In [16]:
adata.layers["counts"] = adata.X.copy()

In [17]:
sc.settings.seed = 42
sc.pp.normalize_total(adata, target_sum=1e4)
sc.pp.log1p(adata)
adata.layers["logcounts"] = adata.X.copy()
sc.pp.highly_variable_genes(adata, n_top_genes=2000, flavor="cell_ranger", batch_key="orig.ident")
sc.tl.pca(adata, n_comps=30, use_highly_variable=True)

In [18]:
adata

AnnData object with n_obs × n_vars = 160667 × 44223
    obs: 'orig.ident', 'nCount_RNA', 'nFeature_RNA', 'sample', 'stage', 'percent.mt', 'species', 'embryo', 'platform', 'anno', 'sub_anno', 'harmo.anno', 'doublet', 'doublet_score'
    var: 'features', 'n_cells', 'highly_variable', 'means', 'dispersions', 'dispersions_norm', 'highly_variable_nbatches', 'highly_variable_intersection'
    uns: 'log1p', 'hvg', 'pca'
    obsm: 'X_pca'
    varm: 'PCs'
    layers: 'counts', 'logcounts'

In [19]:
adata_hvg = adata[:, adata.var.highly_variable].copy()

In [20]:
adata_hvg

AnnData object with n_obs × n_vars = 160667 × 2000
    obs: 'orig.ident', 'nCount_RNA', 'nFeature_RNA', 'sample', 'stage', 'percent.mt', 'species', 'embryo', 'platform', 'anno', 'sub_anno', 'harmo.anno', 'doublet', 'doublet_score'
    var: 'features', 'n_cells', 'highly_variable', 'means', 'dispersions', 'dispersions_norm', 'highly_variable_nbatches', 'highly_variable_intersection'
    uns: 'log1p', 'hvg', 'pca'
    obsm: 'X_pca'
    varm: 'PCs'
    layers: 'counts', 'logcounts'

In [ ]:
####Unintegrated
adata.obsm["Unintegrated"] = adata_hvg.obsm["X_pca"]
adata_hvg.obsm["Unintegrated"] = adata_hvg.obsm["X_pca"]
sc.pp.neighbors(adata, use_rep="Unintegrated",random_state=42)
sc.tl.leiden(adata, resolution=0.5,key_added=f"Unintegrated_res_0.5",random_state=42)
sc.tl.umap(adata,random_state=42)
adata.obsm['X_Unintegrated'] = adata.obsm['X_umap']
sc.pl.umap(adata, color=['orig.ident'], save="Unintegrated_orig_ident.pdf")
sc.pl.umap(adata, color=['Unintegrated_res_0.5'], save="Unintegrated_res_0.5.pdf")
sc.pl.umap(adata, color=['stage'], save="Unintegrated_stage.pdf")
sc.pl.umap(adata, color=['harmo.anno'], save="Unintegrated_harmo.anno.pdf")

In [22]:
adata

AnnData object with n_obs × n_vars = 160667 × 44223
    obs: 'orig.ident', 'nCount_RNA', 'nFeature_RNA', 'sample', 'stage', 'percent.mt', 'species', 'embryo', 'platform', 'anno', 'sub_anno', 'harmo.anno', 'doublet', 'doublet_score', 'Unintegrated_res_0.5'
    var: 'features', 'n_cells', 'highly_variable', 'means', 'dispersions', 'dispersions_norm', 'highly_variable_nbatches', 'highly_variable_intersection'
    uns: 'log1p', 'hvg', 'pca', 'neighbors', 'leiden', 'umap', 'orig.ident_colors', 'Unintegrated_res_0.5_colors', 'stage_colors', 'harmo.anno_colors'
    obsm: 'X_pca', 'Unintegrated', 'X_umap', 'X_Unintegrated'
    varm: 'PCs'
    layers: 'counts', 'logcounts'
    obsp: 'distances', 'connectivities'

In [23]:
###Scanorama
import scanorama
batch_cats = adata_hvg.obs['orig.ident'].cat.categories
adata_list = [adata_hvg[adata_hvg.obs['orig.ident'] == b].copy() for b in batch_cats]
scanorama.integrate_scanpy(adata_list)

adata.obsm["Scanorama"] = np.zeros((adata_hvg.shape[0], adata_list[0].obsm["X_scanorama"].shape[1]))
for i, b in enumerate(batch_cats):
    adata.obsm["Scanorama"][adata_hvg.obs['orig.ident'] == b] = adata_list[i].obsm["X_scanorama"]

Found 2000 genes among all datasets
[[0.         0.00344116 0.         0.31833333 0.17467587 0.28212876
  0.26494565]
 [0.         0.         0.62307692 0.02333333 0.06262904 0.01514109
  0.00275292]
 [0.         0.         0.         0.41538462 0.17179487 0.
  0.04871795]
 [0.         0.         0.         0.         0.47166667 0.155
  0.39      ]
 [0.         0.         0.         0.         0.         0.16042056
  0.0263587 ]
 [0.         0.         0.         0.         0.         0.
  0.3076087 ]
 [0.         0.         0.         0.         0.         0.
  0.        ]]
Processing datasets (1, 2)
Processing datasets (3, 4)
Processing datasets (2, 3)
Processing datasets (3, 6)
Processing datasets (0, 3)
Processing datasets (5, 6)
Processing datasets (0, 5)
Processing datasets (0, 6)
Processing datasets (0, 4)
Processing datasets (2, 4)
Processing datasets (4, 5)
Processing datasets (3, 5)


In [ ]:
adata_hvg.obsm["Scanorama"] = adata.obsm["Scanorama"]
sc.pp.neighbors(adata, use_rep="Scanorama",random_state=42)
sc.tl.leiden(adata, resolution=0.5, key_added=f"Scanorama_res_0.5",random_state=42)
sc.tl.umap(adata,random_state=42)
adata.obsm['X_Scanorama'] = adata.obsm['X_umap']
adata_hvg.obsm['X_Scanorama'] = adata.obsm['X_umap']
sc.pl.umap(adata, color=['orig.ident'], save="Scanorama_orig_ident.pdf")
sc.pl.umap(adata, color=['Scanorama_res_0.5'], save="Scanorama_res_0.5.pdf")
sc.pl.umap(adata, color=['stage'], save="Scanorama_stage.pdf")
sc.pl.umap(adata, color=['harmo.anno'], save="Scanorama_harmo.anno.pdf")

In [25]:
####Liger
bdata = adata_hvg.copy()
# Pyliger normalizes by library size with a size factor of 1
# So here we give it the count data
bdata.X = bdata.layers["counts"]
# List of adata per batch
adata_list = [bdata[bdata.obs['orig.ident'] == b].copy() for b in batch_cats]
for i, ad in enumerate(adata_list):
    ad.uns["sample_name"] = batch_cats[i]
    # Hack to make sure each method uses the same genes
    ad.uns["var_gene_idx"] = np.arange(bdata.n_vars)

liger_data = pyliger.create_liger(adata_list, remove_missing=False, make_sparse=False)
# Hack to make sure each method uses the same genes
liger_data.var_genes = bdata.var_names
pyliger.normalize(liger_data)
pyliger.scale_not_center(liger_data)
pyliger.optimize_ALS(liger_data, k=30)
pyliger.quantile_norm(liger_data)

adata.obsm["LIGER"] = np.zeros((adata_hvg.shape[0], liger_data.adata_list[0].obsm["H_norm"].shape[1]))
for i, b in enumerate(batch_cats):
    adata.obsm["LIGER"][adata_hvg.obs['orig.ident'] == b] = liger_data.adata_list[i].obsm["H_norm"]

/home/lanxiang/miniconda3/envs/sc/lib/python3.8/site-packages/pyliger/preprocessing/_scale.py:88: RuntimeWarning: divide by zero encountered in divide
100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████| 30/30 [23:45<00:00, 47.53s/it]


In [ ]:
adata_hvg.obsm["LIGER"] = adata.obsm["LIGER"]
sc.pp.neighbors(adata, use_rep="LIGER",random_state=42)
sc.tl.leiden(adata, resolution=0.5, key_added=f"LIGER_res_0.5",random_state=42)
sc.tl.umap(adata,random_state=42)
adata.obsm['X_LIGER'] = adata.obsm['X_umap']
adata_hvg.obsm['X_LIGER'] = adata.obsm['X_LIGER']
sc.pl.umap(adata, color=['orig.ident'],save="Liger_orig.ident.pdf")
sc.pl.umap(adata, color=['LIGER_res_0.5'],save="Liger_res_0.5.pdf")
sc.pl.umap(adata, color=['stage'],save="Liger_stage.pdf")
sc.pl.umap(adata, color=['harmo.anno'],save="Liger_harmo.anno.pdf")

In [27]:
###harmony
adata.obsm["Harmony"] = harmonize(adata_hvg.obsm["X_pca"], adata_hvg.obs, batch_key="orig.ident")

	Initialization is completed.
	Completed 1 / 10 iteration(s).
	Completed 2 / 10 iteration(s).
	Completed 3 / 10 iteration(s).
Reach convergence after 3 iteration(s).


In [ ]:
adata_hvg.obsm["Harmony"] = adata.obsm["Harmony"]
sc.pp.neighbors(adata, use_rep="Harmony",random_state=42)
sc.tl.leiden(adata, resolution=0.5, key_added=f"Harmony_res_0.5",random_state=42)
sc.tl.umap(adata,random_state=42)
adata.obsm['X_Harmony'] = adata.obsm['X_umap']
adata_hvg.obsm['X_Harmony'] = adata.obsm['X_Harmony']
sc.pl.umap(adata, color=['orig.ident'],save="Harmony_orig.ident.pdf")
sc.pl.umap(adata, color=['Harmony_res_0.5'],save="Harmony_res_0.5.pdf")
sc.pl.umap(adata, color=['stage'],save="Harmony_stage.pdf")
sc.pl.umap(adata, color=['harmo.anno'],save="Harmony_harmo.anno.pdf")

In [29]:
print(adata.layers["counts"])

(0, 0)        10.4088
  (0, 2)        11.8958
  (0, 3)        34.2003
  (0, 26)       7.43484
  (0, 38)       92.1921
  (0, 39)       13.3827
  (0, 41)       5.94788
  (0, 42)       2.97394
  (0, 43)       5.94788
  (0, 44)       2.97394
  (0, 46)       72.8615
  (0, 48)       26.7654
  (0, 49)       5.94788
  (0, 51)       17.8436
  (0, 52)       52.0439
  (0, 54)       20.8176
  (0, 57)       132.34
  (0, 58)       8.92181
  (0, 59)       303.342
  (0, 63)       196.28
  (0, 64)       92.1921
  (0, 67)       89.2181
  (0, 72)       4.46091
  (0, 74)       72.8615
  (0, 76)       59.4788
  :     :
  (160666, 44028)       29.0
  (160666, 44037)       15.0
  (160666, 44038)       696.0
  (160666, 44042)       115.0
  (160666, 44045)       23.0
  (160666, 44073)       44.0
  (160666, 44084)       1.0
  (160666, 44090)       1.0
  (160666, 44094)       45.0
  (160666, 44102)       3.0
  (160666, 44104)       2.0
  (160666, 44111)       35.0
  (160666, 44115)       26.0
  (160666, 44121)       1.0
  (160666, 44139)       1.0
  (160666, 44143)       67.0
  (160666, 44153)       1.0
  (160666, 44171)       5.0
  (160666, 44184)       2.0
  (160666, 44186)       9.0
  (160666, 44191)       1.0
  (160666, 44199)       1.0
  (160666, 44207)       23.0
  (160666, 44211)       85.0
  (160666, 44218)       14.0

In [30]:
###scVI
scvi.model.SCVI.setup_anndata(adata_hvg, layer="counts", batch_key="orig.ident")
vae = scvi.model.SCVI(adata_hvg, gene_likelihood="nb", n_layers=2, n_latent=30)
vae.train()
adata.obsm["scVI"] = vae.get_latent_representation()

No GPU/TPU found, falling back to CPU. (Set TF_CPP_MIN_LOG_LEVEL=0 and rerun for more info.)
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


Epoch 50/50: 100%|██████████████████████████████████████████████████████████████████████████| 50/50 [12:39<00:00, 14.08s/it, loss=1.56e+03, v_num=1]

`Trainer.fit` stopped: `max_epochs=50` reached.


Epoch 50/50: 100%|██████████████████████████████████████████████████████████████████████████| 50/50 [12:39<00:00, 15.20s/it, loss=1.56e+03, v_num=1]


In [ ]:
adata_hvg.obsm["scVI"] = adata.obsm["scVI"]
sc.pp.neighbors(adata, use_rep="scVI",random_state=42)
sc.tl.leiden(adata, resolution=0.5,key_added=f"scVI_res_0.5",random_state=42)
sc.tl.umap(adata,random_state=42)
adata.obsm['X_scVI'] = adata.obsm['X_umap']
adata_hvg.obsm['X_scVI'] = adata.obsm['X_scVI']
sc.pl.umap(adata, color=['orig.ident'],save="scVI_orig.ident.pdf")
sc.pl.umap(adata, color=['scVI_res_0.5'],save="scVI_res_0.5.pdf")
sc.pl.umap(adata, color=['stage'],save="scVI_stage.pdf")
sc.pl.umap(adata, color=['harmo.anno'],save="scVI_harmo.anno.pdf")

In [32]:
###scANVI
lvae = scvi.model.SCANVI.from_scvi_model(
    vae,
    adata=adata_hvg,
    labels_key="harmo.anno",
    unlabeled_category="Unknown",
)
lvae.train(max_epochs=20, n_samples_per_label=100)
adata.obsm["scANVI"] = lvae.get_latent_representation()

INFO     Training for 20 epochs.                                                                                   


GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


Epoch 20/20: 100%|██████████████████████████████████████████████████████████████████████████| 20/20 [11:02<00:00, 33.46s/it, loss=1.64e+03, v_num=1]

`Trainer.fit` stopped: `max_epochs=20` reached.


Epoch 20/20: 100%|██████████████████████████████████████████████████████████████████████████| 20/20 [11:02<00:00, 33.12s/it, loss=1.64e+03, v_num=1]


In [ ]:
adata_hvg.obsm["scANVI"] = adata.obsm["scANVI"]
sc.pp.neighbors(adata, use_rep="scANVI",random_state=42)
sc.tl.leiden(adata, resolution=0.5,key_added=f"scANVI_res_0.5",random_state=42)
sc.tl.umap(adata,random_state=42)
adata.obsm['X_scANVI'] = adata.obsm['X_umap']
adata_hvg.obsm['X_scANVI'] = adata.obsm['X_scANVI']
sc.pl.umap(adata, color=['orig.ident'],save="scANVI_orig.ident.pdf")
sc.pl.umap(adata, color=['scANVI_res_0.5'],save="scANVI_res_0.5.pdf")
sc.pl.umap(adata, color=['stage'],save="scANVI_stage.pdf")
sc.pl.umap(adata, color=['harmo.anno'],save="scANVI_harmo.anno.pdf")

In [34]:
####CCA
adata_seurat = adata_hvg.copy()

adata_seurat.obs["orig.ident"] = adata_seurat.obs["orig.ident"].astype(str)
adata_seurat.obs["harmo.anno"] = adata_seurat.obs["harmo.anno"].astype(str)

del adata_seurat.uns
adata_seurat

AnnData object with n_obs × n_vars = 160667 × 2000
    obs: 'orig.ident', 'nCount_RNA', 'nFeature_RNA', 'sample', 'stage', 'percent.mt', 'species', 'embryo', 'platform', 'anno', 'sub_anno', 'harmo.anno', 'doublet', 'doublet_score', '_scvi_batch', '_scvi_labels'
    var: 'features', 'n_cells', 'highly_variable', 'means', 'dispersions', 'dispersions_norm', 'highly_variable_nbatches', 'highly_variable_intersection'
    obsm: 'X_pca', 'Unintegrated', 'Scanorama', 'X_Scanorama', 'LIGER', 'X_LIGER', 'Harmony', 'X_Harmony', 'scVI', 'X_scVI', 'scANVI', 'X_scANVI'
    varm: 'PCs'
    layers: 'counts', 'logcounts'

In [35]:
#####to R
%R -i adata_seurat
adata_seurat

In addition: Warning messages:
1: In (function (package, help, pos = 2, lib.loc = NULL, character.only = FALSE,  :
  library ‘/usr/lib/R/site-library’ contains no packages
2: In (function (package, help, pos = 2, lib.loc = NULL, character.only = FALSE,  :
  library ‘/usr/lib/R/site-library’ contains no packages
3: In (function (package, help, pos = 2, lib.loc = NULL, character.only = FALSE,  :
  library ‘/usr/lib/R/site-library’ contains no packages
4: In (function (package, help, pos = 2, lib.loc = NULL, character.only = FALSE,  :
  library ‘/usr/lib/R/site-library’ contains no packages


AnnData object with n_obs × n_vars = 160667 × 2000
    obs: 'orig.ident', 'nCount_RNA', 'nFeature_RNA', 'sample', 'stage', 'percent.mt', 'species', 'embryo', 'platform', 'anno', 'sub_anno', 'harmo.anno', 'doublet', 'doublet_score', '_scvi_batch', '_scvi_labels'
    var: 'features', 'n_cells', 'highly_variable', 'means', 'dispersions', 'dispersions_norm', 'highly_variable_nbatches', 'highly_variable_intersection'
    obsm: 'X_pca', 'Unintegrated', 'Scanorama', 'X_Scanorama', 'LIGER', 'X_LIGER', 'Harmony', 'X_Harmony', 'scVI', 'X_scVI', 'scANVI', 'X_scANVI'
    varm: 'PCs'
    layers: 'counts', 'logcounts'

In [36]:
%%R -i adata_seurat
library(Seurat)
seurat <- as.Seurat(adata_seurat, counts = "counts", data = "logcounts")
seurat

An object of class Seurat 
2000 features across 160667 samples within 1 assay 
Active assay: originalexp (2000 features, 0 variable features)
 2 layers present: counts, data
 12 dimensional reductions calculated: PCA, Unintegrated, Scanorama, X_Scanorama, LIGER, X_LIGER, Harmony, X_Harmony, scVI, X_scVI, scANVI, X_scANVI


In [37]:
label_key = "harmo.anno"
batch_key = "orig.ident"

In [38]:
%%R -i batch_key
batch_list <- SplitObject(seurat, split.by = "orig.ident")
batch_list

$Nakamura
An object of class Seurat 
2000 features across 390 samples within 1 assay 
Active assay: originalexp (2000 features, 0 variable features)
 2 layers present: counts, data
 12 dimensional reductions calculated: PCA, Unintegrated, Scanorama, X_Scanorama, LIGER, X_LIGER, Harmony, X_Harmony, scVI, X_scVI, scANVI, X_scANVI

$Zhai_2022
An object of class Seurat 
2000 features across 64575 samples within 1 assay 
Active assay: originalexp (2000 features, 0 variable features)
 2 layers present: counts, data
 12 dimensional reductions calculated: PCA, Unintegrated, Scanorama, X_Scanorama, LIGER, X_LIGER, Harmony, X_Harmony, scVI, X_scVI, scANVI, X_scANVI

$Ma
An object of class Seurat 
2000 features across 1453 samples within 1 assay 
Active assay: originalexp (2000 features, 0 variable features)
 2 layers present: counts, data
 12 dimensional reductions calculated: PCA, Unintegrated, Scanorama, X_Scanorama, LIGER, X_LIGER, Harmony, X_Harmony, scVI, X_scVI, scANVI, X_scANVI

$Niu
An o

In [39]:
%%R
anchors <- FindIntegrationAnchors(batch_list, anchor.features = rownames(seurat))
anchors

  |                                                  | 0 % ~calculating   |++++++++                                          | 14% ~00s           |+++++++++++++++                                   | 29% ~13s           |++++++++++++++++++++++                            | 43% ~07s           |+++++++++++++++++++++++++++++                     | 57% ~04s           |++++++++++++++++++++++++++++++++++++              | 71% ~03s           |+++++++++++++++++++++++++++++++++++++++++++       | 86% ~02s           |++++++++++++++++++++++++++++++++++++++++++++++++++| 100% elapsed=13s  
  |                                                  | 0 % ~calculating   |+++                                               | 5 % ~15m 50s       |+++++                                             | 10% ~07m 57s       |++++++++                                          | 14% ~54m 32s       |++++++++++                                        | 19% ~39m 10s       |++++++++++++                                      | 24% ~47

Scaling features for provided objects
Finding all pairwise anchors
Running CCA
Merging objects
Finding neighborhoods
Finding anchors
	Found 1929 anchors
Filtering anchors
	Retained 208 anchors
Running CCA
Merging objects
Finding neighborhoods
Finding anchors
	Found 1573 anchors
Filtering anchors
	Retained 1048 anchors
Running CCA
Merging objects
Finding neighborhoods
Finding anchors
	Found 5899 anchors
Filtering anchors
	Retained 4179 anchors
Running CCA
Merging objects
Finding neighborhoods
Finding anchors
	Found 1333 anchors
Filtering anchors
	Retained 1101 anchors
Running CCA
Merging objects
Finding neighborhoods
Finding anchors
	Found 2842 anchors
Filtering anchors
	Retained 2379 anchors
Running CCA
Merging objects
Finding neighborhoods
Finding anchors
	Found 2143 anchors
Filtering anchors
	Retained 1989 anchors
Running CCA
Merging objects
Finding neighborhoods
Finding anchors
	Found 1820 anchors
Filtering anchors
	Retained 717 anchors
Running CCA
Merging objects
Finding neighborho

In [40]:
%%R
integrated <- IntegrateData(anchors)
integrated

An object of class Seurat 
4000 features across 160667 samples within 2 assays 
Active assay: integrated (2000 features, 2000 variable features)
 1 layer present: data
 1 other assay present: originalexp


Merging dataset 4 into 2
Extracting anchors for merged samples
Finding integration vectors
Finding integration vector weights
0%   10   20   30   40   50   60   70   80   90   100%
[----|----|----|----|----|----|----|----|----|----|
**************************************************|
Integrating data
Merging dataset 7 into 6
Extracting anchors for merged samples
Finding integration vectors
Finding integration vector weights
0%   10   20   30   40   50   60   70   80   90   100%
[----|----|----|----|----|----|----|----|----|----|
**************************************************|
Integrating data
Merging dataset 3 into 2 4
Extracting anchors for merged samples
Finding integration vectors
Finding integration vector weights
0%   10   20   30   40   50   60   70   80   90   100%
[----|----|----|----|----|----|----|----|----|----|
**************************************************|
Integrating data
Merging dataset 1 into 5
Extracting anchors for merged samples
Finding integration vectors
F

In [41]:
%%R
integrated

An object of class Seurat 
4000 features across 160667 samples within 2 assays 
Active assay: integrated (2000 features, 2000 variable features)
 1 layer present: data
 1 other assay present: originalexp


In [42]:
%%R -o integrated_expr
integrated_expr <- GetAssayData(integrated)

integrated_expr <- integrated_expr[rownames(seurat), colnames(seurat)]

integrated_expr <- t(integrated_expr)
print(integrated_expr[1:10, 1:10])

10 x 10 sparse Matrix of class "dgCMatrix"
                                                                      
E13_MS10T45_1_1 0.6667583 0.04934495 0.2398668  0.17903400  0.11350276
E13_MS10T46_1_1 0.5565302 0.00824349 0.1832880 -0.01924275 -0.12075684
E13_MS10T48_1_1 1.0172319 0.05879730 0.2449456 -0.06301326  0.12344772
E13_MS10T49_1_1 0.7218265 0.04218439 0.1859685  0.26752462 -0.05500853
E13_MS13T66_1_1 0.6406536 0.04907167 0.2367585  0.20997918  0.46285080
E13_MS13T67_1_1 0.6059875 0.06462054 0.2690634 -0.10038470  0.01444083
E13_MS13T68_1_1 0.6207846 0.05700810 0.2571653  0.31183046  0.25380806
E13_MS13T69_1_1 0.9305673 0.07307878 0.2662938 -0.01423268  0.12064346
E13_MS13T70_1_1 0.6216592 0.05275685 0.2593982  0.37094252  0.03792298
E13_MS13T71_1_1 0.5697991 0.04919405 0.2325725  0.13434759  0.54675845
                                                                        
E13_MS10T45_1_1 1.0998358 -0.007173378  0.38810258 0.18174656 -0.1877693
E13_MS10T46_1_1 1.2875444 -0.0

  [[ suppressing 10 column names ‘KIF26B’, ‘PLD5’, ‘RYR2’ ... ]]


In [43]:
## save to adata key
adata_seurat.X = integrated_expr
adata_seurat.layers["seurat"] = integrated_expr
print(adata_seurat)
adata.X

AnnData object with n_obs × n_vars = 160667 × 2000
    obs: 'orig.ident', 'nCount_RNA', 'nFeature_RNA', 'sample', 'stage', 'percent.mt', 'species', 'embryo', 
'platform', 'anno', 'sub_anno', 'harmo.anno', 'doublet', 'doublet_score', '_scvi_batch', '_scvi_labels'
    var: 'features', 'n_cells', 'highly_variable', 'means', 'dispersions', 'dispersions_norm', 
'highly_variable_nbatches', 'highly_variable_intersection'
    obsm: 'X_pca', 'Unintegrated', 'Scanorama', 'X_Scanorama', 'LIGER', 'X_LIGER', 'Harmony', 'X_Harmony', 'scVI', 
'X_scVI', 'scANVI', 'X_scANVI'
    varm: 'PCs'
    layers: 'counts', 'logcounts', 'seurat'

<160667x44223 sparse matrix of type '<class 'numpy.float64'>'
	with 842301664 stored elements in Compressed Sparse Row format>

In [44]:
sc.tl.pca(adata_seurat)
adata.obsm["CCA"] = adata_seurat.obsm["X_pca"]
adata_hvg.obsm["CCA"] = adata_seurat.obsm["X_pca"]

In [ ]:
sc.pp.neighbors(adata, use_rep="CCA",random_state=42)
sc.tl.leiden(adata, resolution=0.5, key_added=f"CCA_res_0.5",random_state=42)
sc.tl.umap(adata,random_state=42)
adata.obsm['X_CCA'] = adata.obsm['X_umap']
adata_hvg.obsm['X_CCA'] = adata.obsm['X_CCA']
sc.pl.umap(adata, color=['orig.ident'],save="CCA_orig.ident.pdf")
sc.pl.umap(adata, color=['CCA_res_0.5'],save="CCA_res_0.5.pdf")
sc.pl.umap(adata, color=['stage'],save="CCA_stage.pdf")
sc.pl.umap(adata, color=['harmo.anno'],save="CCA_harmo.anno.pdf")

In [46]:
###fastmnn
adata_fastmnn = adata_hvg.copy()
adata_fastmnn

AnnData object with n_obs × n_vars = 160667 × 2000
    obs: 'orig.ident', 'nCount_RNA', 'nFeature_RNA', 'sample', 'stage', 'percent.mt', 'species', 'embryo', 'platform', 'anno', 'sub_anno', 'harmo.anno', 'doublet', 'doublet_score', '_scvi_batch', '_scvi_labels'
    var: 'features', 'n_cells', 'highly_variable', 'means', 'dispersions', 'dispersions_norm', 'highly_variable_nbatches', 'highly_variable_intersection'
    uns: 'log1p', 'hvg', 'pca', '_scvi_uuid', '_scvi_manager_uuid'
    obsm: 'X_pca', 'Unintegrated', 'Scanorama', 'X_Scanorama', 'LIGER', 'X_LIGER', 'Harmony', 'X_Harmony', 'scVI', 'X_scVI', 'scANVI', 'X_scANVI', 'CCA', 'X_CCA'
    varm: 'PCs'
    layers: 'counts', 'logcounts'

In [47]:
adata_fastmnn.obs["orig.ident"] = adata_fastmnn.obs["orig.ident"].astype(str)
adata_fastmnn.obs["harmo.anno"] = adata_fastmnn.obs["harmo.anno"].astype(str)

del adata_fastmnn.uns
adata_fastmnn

AnnData object with n_obs × n_vars = 160667 × 2000
    obs: 'orig.ident', 'nCount_RNA', 'nFeature_RNA', 'sample', 'stage', 'percent.mt', 'species', 'embryo', 'platform', 'anno', 'sub_anno', 'harmo.anno', 'doublet', 'doublet_score', '_scvi_batch', '_scvi_labels'
    var: 'features', 'n_cells', 'highly_variable', 'means', 'dispersions', 'dispersions_norm', 'highly_variable_nbatches', 'highly_variable_intersection'
    obsm: 'X_pca', 'Unintegrated', 'Scanorama', 'X_Scanorama', 'LIGER', 'X_LIGER', 'Harmony', 'X_Harmony', 'scVI', 'X_scVI', 'scANVI', 'X_scANVI', 'CCA', 'X_CCA'
    varm: 'PCs'
    layers: 'counts', 'logcounts'

In [48]:
#####to R
%R -i adata_fastmnn
adata_fastmnn

AnnData object with n_obs × n_vars = 160667 × 2000
    obs: 'orig.ident', 'nCount_RNA', 'nFeature_RNA', 'sample', 'stage', 'percent.mt', 'species', 'embryo', 'platform', 'anno', 'sub_anno', 'harmo.anno', 'doublet', 'doublet_score', '_scvi_batch', '_scvi_labels'
    var: 'features', 'n_cells', 'highly_variable', 'means', 'dispersions', 'dispersions_norm', 'highly_variable_nbatches', 'highly_variable_intersection'
    obsm: 'X_pca', 'Unintegrated', 'Scanorama', 'X_Scanorama', 'LIGER', 'X_LIGER', 'Harmony', 'X_Harmony', 'scVI', 'X_scVI', 'scANVI', 'X_scANVI', 'CCA', 'X_CCA'
    varm: 'PCs'
    layers: 'counts', 'logcounts'

In [49]:
%%R -i adata_fastmnn
library(Seurat)
fastmnn <- as.Seurat(adata_fastmnn, counts = "counts", data = "logcounts")
fastmnn

An object of class Seurat 
2000 features across 160667 samples within 1 assay 
Active assay: originalexp (2000 features, 0 variable features)
 2 layers present: counts, data
 14 dimensional reductions calculated: PCA, Unintegrated, Scanorama, X_Scanorama, LIGER, X_LIGER, Harmony, X_Harmony, scVI, X_scVI, scANVI, X_scANVI, CCA, X_CCA


In [50]:
label_key = "harmo.anno"
batch_key = "orig.ident"

In [51]:
%%R -i batch_key
batch_list <- SplitObject(fastmnn, split.by = "orig.ident")
batch_list

$Nakamura
An object of class Seurat 
2000 features across 390 samples within 1 assay 
Active assay: originalexp (2000 features, 0 variable features)
 2 layers present: counts, data
 14 dimensional reductions calculated: PCA, Unintegrated, Scanorama, X_Scanorama, LIGER, X_LIGER, Harmony, X_Harmony, scVI, X_scVI, scANVI, X_scANVI, CCA, X_CCA

$Zhai_2022
An object of class Seurat 
2000 features across 64575 samples within 1 assay 
Active assay: originalexp (2000 features, 0 variable features)
 2 layers present: counts, data
 14 dimensional reductions calculated: PCA, Unintegrated, Scanorama, X_Scanorama, LIGER, X_LIGER, Harmony, X_Harmony, scVI, X_scVI, scANVI, X_scANVI, CCA, X_CCA

$Ma
An object of class Seurat 
2000 features across 1453 samples within 1 assay 
Active assay: originalexp (2000 features, 0 variable features)
 2 layers present: counts, data
 14 dimensional reductions calculated: PCA, Unintegrated, Scanorama, X_Scanorama, LIGER, X_LIGER, Harmony, X_Harmony, scVI, X_scVI, scA

In [52]:
%%R
library(Seurat)
library(tidyverse)
library(patchwork)
library(SeuratWrappers)
fastmnn_test <- RunFastMNN(object.list = batch_list)
fastmnn_test

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.5.0     ✔ tibble    3.2.1
✔ lubridate 1.9.3     ✔ tidyr     1.3.1
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ lubridate::%within%() masks IRanges::%within%()
✖ dplyr::collapse()     masks IRanges::collapse()
✖ dplyr::combine()      masks Biobase::combine(), BiocGenerics::combine()
✖ dplyr::count()        masks matrixStats::count()
✖ dplyr::desc()         masks IRanges::desc()
✖ tidyr::expand()       masks Matrix::expand(), S4Vectors::expand()
✖ dplyr::filter()       masks stats::filter()
✖ dplyr::first()        masks S4Vectors::first()
✖ dplyr::lag()          masks stats::lag()
✖ tidyr::pack()         masks Matrix::pack()
✖ ggplot2::Position()   masks BiocGenerics::Position(), base::Position()
✖ purrr::reduce()       masks GenomicRanges::reduce(), IRang

Computing 2000 integration features
No variable features found for object1 in the object.list. Running FindVariableFeatures ...
Calculating gene variances
0%   10   20   30   40   50   60   70   80   90   100%
[----|----|----|----|----|----|----|----|----|----|
**************************************************|
Calculating feature variances of standardized and clipped values
0%   10   20   30   40   50   60   70   80   90   100%
[----|----|----|----|----|----|----|----|----|----|
**************************************************|
No variable features found for object2 in the object.list. Running FindVariableFeatures ...
Calculating gene variances
0%   10   20   30   40   50   60   70   80   90   100%
[----|----|----|----|----|----|----|----|----|----|
**************************************************|
Calculating feature variances of standardized and clipped values
0%   10   20   30   40   50   60   70   80   90   100%
[----|----|----|----|----|----|----|----|----|----|
************

In [53]:
%%R
fastmnn_test

An object of class Seurat 
4000 features across 160667 samples within 2 assays 
Active assay: originalexp (2000 features, 0 variable features)
 2 layers present: counts, data
 1 other assay present: mnn.reconstructed
 1 dimensional reduction calculated: mnn


In [54]:
%%R -o fastmnn_expr
fastmnn_expr <- GetAssayData(fastmnn_test, assay = "mnn.reconstructed")

fastmnn_expr <- fastmnn_expr[rownames(fastmnn), colnames(fastmnn)]

fastmnn_expr <- t(fastmnn_expr)
print(fastmnn_expr[1:10, 1:10])

10 x 10 sparse Matrix of class "dgCMatrix"
                                                                              
E13_MS10T45_1_1 -0.005001656  2.878095e-04 -0.008739553 0.006702612 0.02444431
E13_MS10T46_1_1 -0.008538606 -8.776049e-04 -0.007584642 0.005903818 0.02335280
E13_MS10T48_1_1 -0.005763724 -8.466320e-04 -0.007652116 0.005794694 0.02461755
E13_MS10T49_1_1 -0.009387452 -1.064435e-04 -0.005464187 0.006269262 0.01983828
E13_MS13T66_1_1 -0.010742164  8.168143e-06 -0.005589340 0.005898746 0.02220069
E13_MS13T67_1_1 -0.007948626 -1.398696e-04 -0.004903757 0.005707800 0.02196407
E13_MS13T68_1_1 -0.005540180 -6.755594e-04 -0.006798039 0.005119124 0.02471964
E13_MS13T69_1_1 -0.004282820 -1.495993e-03 -0.007653990 0.004623036 0.02313484
E13_MS13T70_1_1 -0.009303541 -1.062261e-03 -0.007603347 0.005300486 0.02539109
E13_MS13T71_1_1 -0.009504860 -1.931008e-04 -0.005296080 0.006236873 0.02355503
                                                                               
E13_MS10

  [[ suppressing 10 column names ‘KIF26B’, ‘PLD5’, ‘RYR2’ ... ]]


In [55]:
## save to adata key
adata_fastmnn.X = fastmnn_expr
adata_fastmnn.layers["fastmnn"] = fastmnn_expr
print(adata_fastmnn)
adata.X

AnnData object with n_obs × n_vars = 160667 × 2000
    obs: 'orig.ident', 'nCount_RNA', 'nFeature_RNA', 'sample', 'stage', 'percent.mt', 'species', 'embryo', 
'platform', 'anno', 'sub_anno', 'harmo.anno', 'doublet', 'doublet_score', '_scvi_batch', '_scvi_labels'
    var: 'features', 'n_cells', 'highly_variable', 'means', 'dispersions', 'dispersions_norm', 
'highly_variable_nbatches', 'highly_variable_intersection'
    obsm: 'X_pca', 'Unintegrated', 'Scanorama', 'X_Scanorama', 'LIGER', 'X_LIGER', 'Harmony', 'X_Harmony', 'scVI', 
'X_scVI', 'scANVI', 'X_scANVI', 'CCA', 'X_CCA'
    varm: 'PCs'
    layers: 'counts', 'logcounts', 'fastmnn'

<160667x44223 sparse matrix of type '<class 'numpy.float64'>'
	with 842301664 stored elements in Compressed Sparse Row format>

In [56]:
sc.tl.pca(adata_fastmnn)
adata.obsm["FastMNN"] = adata_fastmnn.obsm["X_pca"]
adata_hvg.obsm["FastMNN"] = adata_fastmnn.obsm["X_pca"]

In [ ]:
sc.pp.neighbors(adata, use_rep="FastMNN",random_state=42)
sc.tl.leiden(adata, resolution=0.5, key_added=f"FastMNN_res_0.5",random_state=42)
sc.tl.umap(adata,random_state=42)
adata.obsm['X_FastMNN'] = adata.obsm['X_umap']
adata_hvg.obsm['X_FastMNN'] = adata.obsm['X_FastMNN']
sc.pl.umap(adata, color=['orig.ident'],save="FastMNN_orig.ident.pdf")
sc.pl.umap(adata, color=['FastMNN_res_0.5'],save="FastMNN_res_0.5.pdf")
sc.pl.umap(adata, color=['stage'],save="FastMNN_stage.pdf")
sc.pl.umap(adata, color=['harmo.anno'],save="FastMNN_harmo.anno.pdf")

In [58]:
adata

AnnData object with n_obs × n_vars = 160667 × 44223
    obs: 'orig.ident', 'nCount_RNA', 'nFeature_RNA', 'sample', 'stage', 'percent.mt', 'species', 'embryo', 'platform', 'anno', 'sub_anno', 'harmo.anno', 'doublet', 'doublet_score', 'Unintegrated_res_0.5', 'Scanorama_res_0.5', 'LIGER_res_0.5', 'Harmony_res_0.5', 'scVI_res_0.5', 'scANVI_res_0.5', 'CCA_res_0.5', 'FastMNN_res_0.5'
    var: 'features', 'n_cells', 'highly_variable', 'means', 'dispersions', 'dispersions_norm', 'highly_variable_nbatches', 'highly_variable_intersection'
    uns: 'log1p', 'hvg', 'pca', 'neighbors', 'leiden', 'umap', 'orig.ident_colors', 'Unintegrated_res_0.5_colors', 'stage_colors', 'harmo.anno_colors', 'Scanorama_res_0.5_colors', 'LIGER_res_0.5_colors', 'Harmony_res_0.5_colors', 'scVI_res_0.5_colors', 'scANVI_res_0.5_colors', 'CCA_res_0.5_colors', 'FastMNN_res_0.5_colors'
    obsm: 'X_pca', 'Unintegrated', 'X_umap', 'X_Unintegrated', 'Scanorama', 'X_Scanorama', 'LIGER', 'X_LIGER', 'Harmony', 'X_Harmony', 'scVI

In [59]:
print(adata.layers["counts"])

(0, 0)        10.4088
  (0, 2)        11.8958
  (0, 3)        34.2003
  (0, 26)       7.43484
  (0, 38)       92.1921
  (0, 39)       13.3827
  (0, 41)       5.94788
  (0, 42)       2.97394
  (0, 43)       5.94788
  (0, 44)       2.97394
  (0, 46)       72.8615
  (0, 48)       26.7654
  (0, 49)       5.94788
  (0, 51)       17.8436
  (0, 52)       52.0439
  (0, 54)       20.8176
  (0, 57)       132.34
  (0, 58)       8.92181
  (0, 59)       303.342
  (0, 63)       196.28
  (0, 64)       92.1921
  (0, 67)       89.2181
  (0, 72)       4.46091
  (0, 74)       72.8615
  (0, 76)       59.4788
  :     :
  (160666, 44028)       29.0
  (160666, 44037)       15.0
  (160666, 44038)       696.0
  (160666, 44042)       115.0
  (160666, 44045)       23.0
  (160666, 44073)       44.0
  (160666, 44084)       1.0
  (160666, 44090)       1.0
  (160666, 44094)       45.0
  (160666, 44102)       3.0
  (160666, 44104)       2.0
  (160666, 44111)       35.0
  (160666, 44115)       26.0
  (160666, 44121)       1.0
  (160666, 44139)       1.0
  (160666, 44143)       67.0
  (160666, 44153)       1.0
  (160666, 44171)       5.0
  (160666, 44184)       2.0
  (160666, 44186)       9.0
  (160666, 44191)       1.0
  (160666, 44199)       1.0
  (160666, 44207)       23.0
  (160666, 44211)       85.0
  (160666, 44218)       14.0

In [60]:
####Perform the benchmark
bm = Benchmarker(
    adata,
    batch_key="orig.ident",
    label_key="harmo.anno",
    embedding_obsm_keys=["Unintegrated", "Scanorama", "LIGER", "Harmony", "scVI", "scANVI","CCA","FastMNN"],
    n_jobs=6,
)
bm.benchmark()

Metrics:  60%|█████████████████████████████████████████▍                           | 6/10 [05:42<02:02, 30.73s/it, Batch correction: kbet_per_label]

INFO     ICM consists of a single batch or is too small. Skip.                                                     
INFO     Node consists of a single batch or is too small. Skip.                                                    



Metrics:   0%|                                                                                                               | 0/10 [00:00<?, ?it/s]
                                                                                                                                                    
Metrics:  60%|███████████████████████████████████████▌                          | 6/10 [1:18:35<27:06, 406.62s/it, Batch correction: kbet_per_label]

INFO     ICM consists of a single batch or is too small. Skip.                                                     
INFO     Node consists of a single batch or is too small. Skip.                                                    



Metrics:   0%|                                                                                                               | 0/10 [00:00<?, ?it/s]
                                                                                                                                                    
Metrics:  60%|█████████████████████████████████████████▍                           | 6/10 [04:00<01:26, 21.65s/it, Batch correction: kbet_per_label]

...


Metrics:   0%|                                                                                                               | 0/10 [00:00<?, ?it/s]
                                                                                                                                                    
Metrics:  60%|████████████████████████████████████████▊                           | 6/10 [43:14<14:38, 219.60s/it, Batch correction: kbet_per_label]

INFO     ICM consists of a single batch or is too small. Skip.                                                     
INFO     Node consists of a single batch or is too small. Skip.                                                    



Metrics:   0%|                                                                                                               | 0/10 [00:00<?, ?it/s]
                                                                                                                                                    
Metrics:  60%|████████████████████████████████████████▊                           | 6/10 [43:10<15:03, 226.00s/it, Batch correction: kbet_per_label]

INFO     ICM consists of a single batch or is too small. Skip.                                                     
INFO     Node consists of a single batch or is too small. Skip.                                                    



Embeddings: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████| 8/8 [3:18:10<00:00, 1486.32s/it]

                                                                                                                                                    

In [ ]:
bm.plot_results_table(save_dir="/mnt/disk5/Lanxiang/Human_integrated/integrated/8_Progress/monkey_ref_all/figures")

In [69]:
from rich import print

df2 = bm.get_results()
print(df2)

Isolated labels        KMeans NMI        KMeans ARI  \
Embedding                                                            
Unintegrated          0.106281          0.293291          0.101369   
Scanorama             0.328192           0.56136          0.423138   
LIGER                      1.0               0.0               0.0   
Harmony                    0.0          0.273595          0.201876   
scVI                  0.649641          0.648362          0.358416   
scANVI                0.538081               1.0               1.0   
CCA                   0.010374          0.662809          0.950838   
FastMNN               0.604323          0.735398           0.86354   
Metric Type   Bio conservation  Bio conservation  Bio conservation   

              Silhouette label             cLISI  Silhouette batch  \
Embedding                                                            
Unintegrated          0.449101               1.0               0.0   
Scanorama             0.857747          0.950061               1.0   
LIGER                   0.5261               0.0          0.700912   
Harmony                    0.0           0.83035          0.921375   
scVI                  0.678633           0.88213          0.923208   
scANVI                     1.0          0.883161          0.838819   
CCA                   0.397611          0.684368          0.786479   
FastMNN               0.970551          0.801949          0.767449   
Metric Type   Bio conservation  Bio conservation  Batch correction   

                         iLISI              KBET Graph connectivity  \
Embedding                                                             
Unintegrated               0.0               0.0           0.775295   
Scanorama             0.000001          0.466107           0.777416   
LIGER                      1.0               1.0                0.0   
Harmony               0.085394          0.576048           0.785512   
scVI                  0.014433          0.460225                1.0   
scANVI                0.029017          0.498464           0.965677   
CCA                   0.498113          0.747074           0.655236   
FastMNN               0.036824          0.476926           0.755993   
Metric Type   Batch correction  Batch correction   Batch correction   

                PCR comparison Batch correction Bio conservation  \
Embedding                                                          
Unintegrated               0.0         0.155059         0.390009   
Scanorama             0.340263         0.516758           0.6241   
LIGER                      1.0         0.740182          0.30522   
Harmony               0.747435         0.623153         0.261164   
scVI                  0.820327         0.643639         0.643436   
scANVI                0.701324          0.60666         0.884248   
CCA                   0.684507         0.674282           0.5412   
FastMNN               0.238395         0.455117         0.795152   
Metric Type   Batch correction  Aggregate score  Aggregate score   

                        Total  
Embedding                      
Unintegrated         0.296029  
Scanorama            0.581163  
LIGER                0.479205  
Harmony               0.40596  
scVI                 0.643517  
scANVI               0.773213  
CCA                  0.594433  
FastMNN              0.659138  
Metric Type   Aggregate score

In [70]:
df2.transpose()

Embedding,Unintegrated,Scanorama,LIGER,Harmony,scVI,scANVI,CCA,FastMNN,Metric Type
Isolated labels,0.106281,0.328192,1.0,0.0,0.649641,0.538081,0.010374,0.604323,Bio conservation
KMeans NMI,0.293291,0.56136,0.0,0.273595,0.648362,1.0,0.662809,0.735398,Bio conservation
KMeans ARI,0.101369,0.423138,0.0,0.201876,0.358416,1.0,0.950838,0.86354,Bio conservation
Silhouette label,0.449101,0.857747,0.5261,0.0,0.678633,1.0,0.397611,0.970551,Bio conservation
cLISI,1.0,0.950061,0.0,0.83035,0.88213,0.883161,0.684368,0.801949,Bio conservation
Silhouette batch,0.0,1.0,0.700912,0.921375,0.923208,0.838819,0.786479,0.767449,Batch correction
iLISI,0.0,0.000001,1.0,0.085394,0.014433,0.029017,0.498113,0.036824,Batch correction
KBET,0.0,0.466107,1.0,0.576048,0.460225,0.498464,0.747074,0.476926,Batch correction
Graph connectivity,0.775295,0.777416,0.0,0.785512,1.0,0.965677,0.655236,0.755993,Batch correction
PCR comparison,0.0,0.340263,1.0,0.747435,0.820327,0.701324,0.684507,0.238395,Batch correction


In [71]:
df2_transposed = df2.transpose()
df2_transposed.to_csv('scib_df_transposed_min_max_scale=True.csv', index=False)

In [ ]:
bm.plot_results_table(min_max_scale=False,save_dir="/mnt/disk5/Lanxiang/Human_integrated/integrated/8_Progress/monkey_ref_all/figures/")

In [63]:
adata.raw.var.rename(columns={'_index': 'index'}, inplace=True)
adata.write_h5ad(filename="scib_monkey_remove_Sasaki8.20.h5ad")

In [64]:
from rich import print

df = bm.get_results(min_max_scale=False)
print(df)

Isolated labels        KMeans NMI        KMeans ARI  \
Embedding                                                            
Unintegrated          0.450284          0.467911          0.240439   
Scanorama             0.494926          0.506873          0.298482   
LIGER                 0.630077          0.425284          0.222153   
Harmony               0.428903          0.465049          0.258569   
scVI                  0.559593          0.519518          0.286807   
scANVI                0.537151          0.570626          0.402541   
CCA                    0.43099          0.521618          0.393672   
FastMNN               0.550477          0.532168          0.377925   
Metric Type   Bio conservation  Bio conservation  Bio conservation   

              Silhouette label             cLISI  Silhouette batch  \
Embedding                                                            
Unintegrated          0.497519          0.998962           0.77526   
Scanorama              0.52551          0.998392           0.87171   
LIGER                 0.502793          0.987538          0.842863   
Harmony               0.466756          0.997024          0.864127   
scVI                  0.513241          0.997616          0.864303   
scANVI                0.535254          0.997628          0.856164   
CCA                   0.493992          0.995357          0.851116   
FastMNN               0.533237            0.9967           0.84928   
Metric Type   Bio conservation  Bio conservation  Batch correction   

                         iLISI              KBET Graph connectivity  \
Embedding                                                             
Unintegrated               0.0            0.1741           0.840131   
Scanorama                  0.0           0.25923           0.840603   
LIGER                 0.063976           0.35674           0.667429   
Harmony               0.005463          0.279309           0.842407   
scVI                  0.000923          0.258155           0.890185   
scANVI                0.001856          0.265139           0.882539   
CCA                   0.031867          0.310545           0.813387   
FastMNN               0.002356          0.261206           0.835831   
Metric Type   Batch correction  Batch correction   Batch correction   

                PCR comparison Batch correction Bio conservation  \
Embedding                                                          
Unintegrated          0.367755         0.431449         0.531023   
Scanorama             0.552723         0.504853         0.564837   
LIGER                 0.911358         0.568473         0.553569   
Harmony               0.774062         0.553074          0.52326   
scVI                  0.813687         0.565451         0.575355   
scANVI                0.748997         0.550939          0.60864   
CCA                   0.739855         0.549354         0.567126   
FastMNN               0.497347         0.489204         0.598101   
Metric Type   Batch correction  Aggregate score  Aggregate score   

                        Total  
Embedding                      
Unintegrated         0.491193  
Scanorama            0.540843  
LIGER                0.559531  
Harmony              0.535186  
scVI                 0.571393  
scANVI                0.58556  
CCA                  0.560017  
FastMNN              0.554542  
Metric Type   Aggregate score

In [68]:
df.to_csv('scib_df.csv')

In [65]:
df.transpose()

Embedding,Unintegrated,Scanorama,LIGER,Harmony,scVI,scANVI,CCA,FastMNN,Metric Type
Isolated labels,0.450284,0.494926,0.630077,0.428903,0.559593,0.537151,0.43099,0.550477,Bio conservation
KMeans NMI,0.467911,0.506873,0.425284,0.465049,0.519518,0.570626,0.521618,0.532168,Bio conservation
KMeans ARI,0.240439,0.298482,0.222153,0.258569,0.286807,0.402541,0.393672,0.377925,Bio conservation
Silhouette label,0.497519,0.52551,0.502793,0.466756,0.513241,0.535254,0.493992,0.533237,Bio conservation
cLISI,0.998962,0.998392,0.987538,0.997024,0.997616,0.997628,0.995357,0.9967,Bio conservation
Silhouette batch,0.77526,0.87171,0.842863,0.864127,0.864303,0.856164,0.851116,0.84928,Batch correction
iLISI,0.0,0.0,0.063976,0.005463,0.000923,0.001856,0.031867,0.002356,Batch correction
KBET,0.1741,0.25923,0.35674,0.279309,0.258155,0.265139,0.310545,0.261206,Batch correction
Graph connectivity,0.840131,0.840603,0.667429,0.842407,0.890185,0.882539,0.813387,0.835831,Batch correction
PCR comparison,0.367755,0.552723,0.911358,0.774062,0.813687,0.748997,0.739855,0.497347,Batch correction


In [67]:
df_transposed = df.transpose()
df_transposed.to_csv('scib_df_transposed.csv', index=False)